In [5]:
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import commonplayerinfo

# Nikola Jokić
career = commonplayerinfo.CommonPlayerInfo(player_id='203497') 

# pandas data frames (optional: pip install pandas)
career.get_data_frames()[0]

# json
career.get_json()

# dictionary
career.get_dict()

{'resource': 'commonplayerinfo',
 'parameters': [{'PlayerID': 203497}, {'LeagueID': None}],
 'resultSets': [{'name': 'CommonPlayerInfo',
   'headers': ['PERSON_ID',
    'FIRST_NAME',
    'LAST_NAME',
    'DISPLAY_FIRST_LAST',
    'DISPLAY_LAST_COMMA_FIRST',
    'DISPLAY_FI_LAST',
    'PLAYER_SLUG',
    'BIRTHDATE',
    'SCHOOL',
    'COUNTRY',
    'LAST_AFFILIATION',
    'HEIGHT',
    'WEIGHT',
    'SEASON_EXP',
    'JERSEY',
    'POSITION',
    'ROSTERSTATUS',
    'GAMES_PLAYED_CURRENT_SEASON_FLAG',
    'TEAM_ID',
    'TEAM_NAME',
    'TEAM_ABBREVIATION',
    'TEAM_CODE',
    'TEAM_CITY',
    'PLAYERCODE',
    'FROM_YEAR',
    'TO_YEAR',
    'DLEAGUE_FLAG',
    'NBA_FLAG',
    'GAMES_PLAYED_FLAG',
    'DRAFT_YEAR',
    'DRAFT_ROUND',
    'DRAFT_NUMBER',
    'GREATEST_75_FLAG'],
   'rowSet': [[203497,
     'Rudy',
     'Gobert',
     'Rudy Gobert',
     'Gobert, Rudy',
     'R. Gobert',
     'rudy-gobert',
     '1992-06-26T00:00:00',
     'Cholet',
     'France',
     'Cholet/France',


In [5]:
import json
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats
import pandas as pd
import pickle

# Assume we have a JSON file 'player_list.json' with the player IDs list
# with open('/home/karolwojtulewicz/code/NSVA/all_players.json', 'r') as file:
#     data = json.load(file)
#     player_ids_with_prefix = data

with open('/home/karolwojtulewicz/code/NSVA/all_unique_players.pickle', 'rb') as file:
    data = pickle.load(file)
    player_ids_with_prefix = [key for key in data.keys() if key.startswith('PLAYER')]

# Step 1: Clean player IDs
player_ids = set(player_id.replace('PLAYER', '').replace(':', '').replace("side", "").replace(",", "") for player_id in player_ids_with_prefix)

# Step 2: Initialize dictionary to store player info
player_info_dict = {}

# Step 3: Iterate over player IDs and fetch data
for player_id in player_ids:
    try:
        career = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
        stats = playercareerstats.PlayerCareerStats(player_id=player_id)
        player_info = career.get_normalized_dict()
        common_info = player_info['commonplayerinfo'][0]  # assuming the first entry is what we want
        player_info = career.get_normalized_dict()
        stats_info = player_info['commonplayerinfo'][0]  # assuming the first entry is what we want

        # Extract required information
        info_dict = {
            'first_name': common_info['FIRST_NAME'],
            'last_name': common_info['LAST_NAME'],
            'height': common_info['HEIGHT'],
            'jersey': common_info['JERSEY']
        }

        # Step 4: Add to the main dictionary
        player_info_dict[player_id] = info_dict

    except Exception as e:
        print(f"Failed to fetch data for Player ID {player_id}: {e}")

# Save the results to a JSON file
with open('player_info_dict.json', 'w') as outfile:
    json.dump(player_info_dict, outfile, indent=4)

print("Finished saving player information to player_info_dict.json")


Failed to fetch data for Player ID 2617: 'commonplayerinfo'


Failed to fetch data for Player ID 201967: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)


In [13]:
from nba_api.stats.endpoints import playbyplayv3

playInfo = playbyplayv3.PlayByPlayV3(game_id='0021900001')
# pandas data frames (optional: pip install pandas)
pbp = playInfo.get_data_frames()[0]

pbp.head(5)

,gameId,actionNumber,clock,period,teamId,teamTricode,personId,playerName,playerNameI,xLegacy,...,isFieldGoal,scoreHome,scoreAway,pointsTotal,location,description,actionType,subType,videoAvailable,actionId
0,0021900001,2,PT12M00.00S,1,0,,0,,,0,...,0,0,0,0,,Start of 1st Period (8:04 PM EST),period,start,0,1
1,0021900001,4,PT12M00.00S,1,1610612761,TOR,201188,Gasol,M. Gasol,0,...,0,,,0,h,Jump Ball Gasol vs. Favors: Tip to Ball,Jump Ball,,1,2
2,0021900001,7,PT11M48.00S,1,1610612740,NOP,1628366,Ball,L. Ball,2,...,1,,,0,v,MISS Ball 11' Driving Floating Jump Shot,Missed Shot,Driving Floating Jump Shot,1,3
3,0021900001,8,PT11M47.00S,1,1610612740,NOP,202324,Favors,D. Favors,0,...,0,,,0,v,Favors REBOUND (Off:1 Def:0),Rebound,Unknown,1,4
4,0021900001,9,PT11M47.00S,1,1610612740,NOP,202324,Favors,D. Favors,0,...,1,0,2,2,v,Favors 1' Tip Layup Shot (2 PTS),Made Shot,Tip Layup Shot,1,5


In [21]:
import json
from itertools import groupby

# Load the dataset and action list
dataset_path = '/home/karolwojtulewicz/code/NSVA/data/ourds_description_only.json'  # Adjust this path
action_list_path = '/home/karolwojtulewicz/code/NSVA/data/action_list.json'  # Adjust this path
coarse_action_list_path = '/home/karolwojtulewicz/code/NSVA/data/action_list_Coarse.json'  # Adjust this path

with open(dataset_path, 'r') as file:
    dataset = json.load(file)

with open(action_list_path, 'r') as file:
    actions_descriptions = json.load(file)
    actions_descriptions = { "action"+str(i+1): action for i, action in enumerate(actions_descriptions)}

with open(coarse_action_list_path, 'r') as file:
    coarse_actions_descriptions = json.load(file)

# Helper function to extract players and actions for each video clip
def extract_players_and_actions(sentences, video_id):
    players, action_codes = [], []
    captionObj = None
    
    for item in sentences:
        if item['video_id'] != video_id:
            continue
        if item['caption'].startswith('<T1>_'):
            players.extend(item['caption'].replace("_"," ").split()[1:])
        elif item['caption'].startswith('<T2>_'):
            action_codes.extend(item['caption'].replace("_"," ").split()[1:])
        elif item['caption'].startswith('<T3>_'):
            captionObj = item
    return list(players), action_codes, captionObj

# Function to generate 10 unique questions and answers based on actions and players
def generate_unique_qa(players, action_codes, actions_descriptions, video_id, captionObj):
    try:
        qa_pairs = []
        if len(players) > 0:
            qa_pairs = qa_pairs + [
                {"question": "What is the first action performed by {}?".format(players[0]), 
             "answer": ("It was a {}".format(" ".join(actions_descriptions[action_codes[0]].replace("_","-").split("-"))))  if len(action_codes) > 0 else "No action performed"},
            
            {"question": "What is last the action performed by {}?".format(players[-1]), 
             "answer": ("It was a {}".format(" ".join(actions_descriptions[action_codes[-1]].replace("_","-").split("-")))) if len(action_codes) > 0 else "No action performed"},
            ]
            if len(action_codes) > 0:
                qa_pairs = qa_pairs + [
                    {"question": "What is the first action in the video?", 
                    "answer": ("{} performed a {}.".format(players[0], " ".join(actions_descriptions[action_codes[0]].replace("_","-").split("-")))) if len(action_codes) > 0 else "No action performed"},
                    
                    {"question": "What is the last action in the video?", 
                    "answer": ("{} performed a {}.".format(players[-1], " ".join(actions_descriptions[action_codes[-1]].replace("_","-").split("-")))) if len(action_codes) > 0 else "No action performed"},
                    
                    {"question": "Was there a foul?", 
                    "answer": ("Yes, {} committed a foul.".format(players[0]) if "Foul" in [coarse_actions_descriptions[actions_descriptions[x]] for x in action_codes]  else "No.") if len(action_codes) > 0 else "No action performed"},
                    
                    {"question": "Was there a shot made?", 
                    "answer": ("Yes." if "Shot" in [coarse_actions_descriptions[actions_descriptions[x]] for x in action_codes] else "No." )if len(action_codes) > 0 else "No action performed"},
                    
                    {"question": "Are there free throws?", 
                    "answer": ("Yes." if "Free_Throw" in [coarse_actions_descriptions[actions_descriptions[x]] for x in action_codes] else "No.") if len(action_codes) > 0 else "No action performed"},
                    
                    {"question": "What are the actions in this clip?", 
                    "answer": (" ; ".join([actions_descriptions[x] for x in action_codes]) if len(action_codes) > 0 else "No action performed")},
                    
                    {"question": "What is the first action?", 
                    "answer": (" ".join([actions_descriptions[x] for x in action_codes][0].replace("_","-").split("-")))  if len(action_codes) > 0 else "No action performed"},
                    
                    {"question": "What is the last action?", 
                    "answer": (" ".join([actions_descriptions[x] for x in action_codes][0].replace("_","-").split("-")))  if len(action_codes) > 0 else "No action performed"},
                    
                    {"question": "How many players are involved in the play?", 
                    "answer": "{} players: {}".format(len(list(set((players)))), ", ".join(list(set(players)))) if len(players) > 0 else "No players involved"},
                    
                    {"question": "Which players are involved in the play?", 
                    "answer": "{}".format(", ".join(list(set(players)))) if len(players) > 0 else "No players involved"},
                    
                    
                ]
        for pair in qa_pairs:
            pair['video_id'] = video_id
            pair['question'] = "<T4>_ " + pair['question'] + " ; " + " ".join(players)
            pair['answer'] = "<T4>_" + pair['answer']
        obj = {"video_id": video_id, "question": "<T3>_ " + " ".join(players), "answer": captionObj['caption']}
        qa_pairs.append(obj)
            
    except Exception as e:
        print(e)
        qa_pairs = []
        obj = {"video_id": video_id, "question": "<T3>_ " + " ".join(players) + " Generate caption", "answer": "<T3>_" +  captionObj['caption']}
        qa_pairs.append(obj)
    

    return qa_pairs

# Process dataset to generate QA pairs for each video clip
qa_by_video_id = {"sentences": []}
sid = 0
for video_id, sentences in groupby(sorted(dataset['sentences'], key=lambda x: x['video_id']), key=lambda x: x['video_id']):
    sentences = list(sentences)
    players, action_codes, captionObj = extract_players_and_actions(sentences, video_id)
    qa_pairs = generate_unique_qa(players, action_codes, actions_descriptions, video_id, captionObj)
    for pair in qa_pairs:
        pair['sid'] = sid
        sid += 1
    qa_by_video_id["sentences"] = qa_by_video_id["sentences"] + qa_pairs

# Output to a new JSON file
output_file_path = 'video_qa_unique_with_answers.json'
with open(output_file_path, 'w') as outfile:
    json.dump(qa_by_video_id, outfile, indent=4)

print(f"Unique QA for videos with answers saved to {output_file_path}")


Unique QA for videos with answers saved to video_qa_unique_with_answers.json


: 